In [1]:
!pip install plotly

  Stored in directory: C:\Users\saqib\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [1]:
!pip install cufflinks

In [2]:
import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
import re

py.init_notebook_mode(connected=False)

In [3]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [4]:
df_customers = pd.read_csv("olist_customers_dataset.csv")
df_orders = pd.read_csv("olist_orders_dataset.csv")
df_order_payments = pd.read_csv("olist_order_payments_dataset.csv")

In [5]:
df_order_payments = df_order_payments.groupby("order_id").agg({"payment_value": "sum"}).reset_index()

df_tmp = pd.merge(df_orders, df_order_payments, on=["order_id"], how="inner")
df = pd.merge(df_tmp, df_customers, on=["customer_id"], how="inner")

cond = df["order_status"] == "delivered"
df = df.loc[cond]

df["order_purchase_date"] = df["order_purchase_timestamp"].str.slice(0, 10)

df["order_purchase_date"] = pd.to_datetime(df["order_purchase_date"], format="%Y-%m-%d")

# Clustering

In [6]:
def clustering_customers(df, date_max, date_min=False, group_range_days=False):
    df = df.copy()

    if(date_min == False):
        cond_f = df["order_purchase_date"] <= pd.to_datetime(date_max)
    else:
        cond_1 = df["order_purchase_date"] <= pd.to_datetime(date_max)
        cond_2 = df["order_purchase_date"] >= pd.to_datetime(date_min)
        cond_f = cond_1 & cond_2

    df = df.loc[cond_f]

    df["today"] = df["order_purchase_date"].max()

    df["today"] = df["today"].dt.date
    df["today"] = pd.to_datetime(df["today"], format="%Y-%m-%d")

    df["order_purchase_days_since"] = df["today"]  - df["order_purchase_date"]
    df["order_purchase_days_since"] = df["order_purchase_days_since"].astype(str)
    df["order_purchase_days_since"] = df["order_purchase_days_since"].str.replace(r'\s+days.*', '', regex=True)
    df["order_purchase_days_since"] = df["order_purchase_days_since"].astype(int)
    df["order_purchase_year"] = df["order_purchase_date"].dt.year

    agg_group = {
        "order_purchase_days_since": ["min", "max", "count"],
        "payment_value": ["sum","mean"]
    }

    df_group = df.groupby(["customer_unique_id"]).agg(agg_group).reset_index()

    df_group.columns = [' '.join(col).strip() for col in df_group.columns.values]
    columns_rename = {
        "order_purchase_days_since min": "first_order_purchase",
        "order_purchase_days_since max": "last_order_purchase",
        "order_purchase_days_since count": "order_purchase_qty",
        "payment_value mean": "payment_value_mean",
        "payment_value sum": "payment_value_sum"
    }

    df_group.rename(columns_rename, axis=1, inplace=True)

    median_payment = df_group["payment_value_mean"].median()

    if(group_range_days == False):
        major_group = 4

        range_days = str(df["order_purchase_date"].max() - df["order_purchase_date"].min()) 
        group_range_days = int(re.sub(r'\s+days.*', '', range_days))/major_group

    cond_payment_zero = df_group['payment_value_mean'] == 0.0

    cond_inactive_1 = df_group['last_order_purchase'] > group_range_days*3
    cond_inactive = cond_inactive_1 | cond_payment_zero

    cond_cold_1 = df_group['last_order_purchase'] > group_range_days*2
    cond_cold_2 = df_group['last_order_purchase'] <= group_range_days*3
    cond_cold = cond_cold_1 & cond_cold_2 & ~(cond_payment_zero)

    cond_hot_1 = df_group['last_order_purchase'] > group_range_days
    cond_hot_2 = df_group['last_order_purchase'] <= group_range_days*2
    cond_hot = cond_hot_1 & cond_hot_2 & ~(cond_payment_zero)

    cond_active_1 = df_group['last_order_purchase'] <= group_range_days
    cond_active = cond_active_1 & ~(cond_payment_zero)

    df_group.loc[cond_inactive, "segment"] = "inactive"
    df_group.loc[cond_cold, "segment"] = "cold"
    df_group.loc[cond_hot, "segment"] = "hot"
    df_group.loc[cond_active, "segment"] = "active"

    cond_hot_high_payment_1 = df_group["segment"] == "hot"
    cond_hot_high_payment_2 = df_group["payment_value_mean"] >= median_payment
    cond_hot_high_payment = cond_hot_high_payment_1 & cond_hot_high_payment_2

    cond_hot_low_payment_1 = df_group["segment"] == "hot"
    cond_hot_low_payment_2 = df_group["payment_value_mean"] < median_payment
    cond_hot_low_payment = cond_hot_low_payment_1 & cond_hot_low_payment_2

    cond_active_high_payment_1 = df_group["segment"] == "active"
    cond_active_high_payment_2 = df_group["payment_value_mean"] >= median_payment
    cond_active_high_payment = cond_active_high_payment_1 & cond_active_high_payment_2

    cond_active_low_payment_1 = df_group["segment"] == "active"
    cond_active_low_payment_2 = df_group["payment_value_mean"] < median_payment
    cond_active_low_payment = cond_active_low_payment_1 & cond_active_low_payment_2

    cond_cold_high_payment_1 = df_group["segment"] == "cold"
    cond_cold_high_payment_2 = df_group["payment_value_mean"] >= median_payment
    cond_cold_high_payment = cond_cold_high_payment_1 & cond_cold_high_payment_2

    cond_cold_low_payment_1 = df_group["segment"] == "cold"
    cond_cold_low_payment_2 = df_group["payment_value_mean"] < median_payment
    cond_cold_low_payment = cond_cold_low_payment_1 & cond_cold_low_payment_2

    df_group["sub_segment"] = "inactive"
    df_group.loc[cond_hot_high_payment, "sub_segment"] = "hot_high_payment_value"
    df_group.loc[cond_hot_low_payment, "sub_segment"] = "hot_low_payment_value"
    df_group.loc[cond_active_high_payment, "sub_segment"] = "active_high_payment_value"
    df_group.loc[cond_active_low_payment, "sub_segment"] = "active_low_payment_value"
    df_group.loc[cond_cold_high_payment, "sub_segment"] = "cold_high_payment_value"
    df_group.loc[cond_cold_low_payment, "sub_segment"] = "cold_low_payment_value"
    
    cond_new_customer = df_group["first_order_purchase"] <= group_range_days*2
    df_group["new_customer"] = 0
    df_group.loc[cond_new_customer, "new_customer"] = 1
    
    return group_range_days, df_group

In [20]:
period_2018, df_clustering_2018 = clustering_customers(df, "2018-12-31")

df_clustering_2018.head(20)

customer_unique_id  first_order_purchase  \
0   0000366f3b9a7992bf8c76cfdf3221e2                   111   
1   0000b849f77a49e4a4ce2b2a4ca5be3f                   114   
2   0000f46a3911fa3c0805444483337064                   537   
3   0000f6ccb0745a6a4b88665a16c9f078                   321   
4   0004aac84e0df4da2b147fca70cf8255                   288   
5   0004bd2a26a76fe21f786e4fbd80607f                   146   
6   00050ab1314c0e55a6ca13cf7181fecf                   131   
7   00053a61a98854899e70ed204dd4bafe                   182   
8   0005e1862207bf6ccc02e4228effd9a0                   543   
9   0005ef4cd20d2893f0d9fbd94d3c0d97                   170   
10  0006fdc98a402fceb4eb0ee528f6a8d4                   407   
11  00082cbe03e478190aadbea78542e933                   283   
12  00090324bbad0e9342388303bb71ba0a                   158   
13  000949456b182f53c18b68d6babc79c1                   128   
14  000a5ad9c4601d2bbdd9ed765d5213b3                   383   
15  000bfa1d2f1a41876493be685390d6d3                   334   
16  000c8bdb58a29e7115cfc257230fb21b                   260   
17  000d460961d6dbfa3ec6c9f5805769e1                   234   
18  000de6019bb59f34c099a907c151d855                   377   
19  000e309254ab1fc5ba99dd469d36bdb4                    65   

    last_order_purchase  order_purchase_qty  payment_value_sum  \
0                   111                   1             141.90   
1                   114                   1              27.19   
2                   537                   1              86.22   
3                   321                   1              43.62   
4                   288                   1             196.89   
5                   146                   1             166.98   
6                   131                   1              35.38   
7                   182                   1             419.18   
8                   543                   1             150.12   
9                   170                   1             129.76   
10                  407                   1              29.00   
11                  283                   1             126.26   
12                  158                   1              63.66   
13                  128                   1              82.05   
14                  383                   1              91.28   
15                  334                   1              46.85   
16                  260                   1              29.00   
17                  234                   1              36.68   
18                  377                   1             257.44   
19                   65                   1              78.42   

    payment_value_mean   segment                sub_segment  new_customer  
0               141.90    active  active_high_payment_value             1  
1                27.19    active   active_low_payment_value             1  
2                86.22  inactive                   inactive             0  
3                43.62       hot      hot_low_payment_value             1  
4               196.89       hot     hot_high_payment_value             1  
5               166.98    active  active_high_payment_value             1  
6                35.38    active   active_low_payment_value             1  
7               419.18       hot     hot_high_payment_value             1  
8               150.12  inactive                   inactive             0  
9               129.76    active  active_high_payment_value             1  
10               29.00      cold     cold_low_payment_value             0  
11              126.26       hot     hot_high_payment_value             1  
12               63.66    active   active_low_payment_value             1  
13               82.05    active   active_low_payment_value             1  
14               91.28      cold     cold_low_payment_value             0  
15               46.85       hot      hot_low_payment_value             1  
16               29.00       hot    

In [8]:
df_revenue_subsegment_2018 = df_clustering_2018.groupby(["sub_segment"]).agg({"payment_value_sum": "sum"}).reset_index()

In [9]:
configure_plotly_browser_state()
trace0 = go.Bar(
    x=df_revenue_subsegment_2018["sub_segment"].values,
    y=df_revenue_subsegment_2018["payment_value_sum"].values,
    marker=dict(
        color=['rgba(36,123,160,1)', 
               'rgba(75,147,177,1)',
               'rgba(112,193,179,1)', 
               'rgba(138,204,192,1)',
               'rgba(243,255,189,1)',
               'rgba(247,255,213,1)',
               'rgba(255,22,84,1)']),
)

data = [trace0]

layout = go.Layout(
    title='Revenue 2018',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# Repurchase Amount by Segment 2018

In [10]:
df_clustering_2018_qty = df_clustering_2018.loc[df_clustering_2018["order_purchase_qty"] > 1]

df_qty_subsegment_2018 = df_clustering_2018_qty.groupby(["sub_segment"]).agg({"order_purchase_qty": "count"}).reset_index()

In [11]:
configure_plotly_browser_state()
trace0 = go.Bar(
    x=df_qty_subsegment_2018["sub_segment"].values,
    y=df_qty_subsegment_2018["order_purchase_qty"].values,
    marker=dict(
        color=['rgba(36,123,160,1)', 
               'rgba(75,147,177,1)',
               'rgba(112,193,179,1)', 
               'rgba(138,204,192,1)',
               'rgba(243,255,189,1)',
               'rgba(247,255,213,1)',
               'rgba(255,22,84,1)']),
)

data = [trace0]

layout = go.Layout(
    title='Repurchase Amount 2018',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# Average days between first and last purchase by segment 2018

In [12]:
df_days_repurchase_subsegment_2018 = df_clustering_2018_qty.groupby(["sub_segment"]).agg({"first_order_purchase": "mean", "last_order_purchase": "mean"}).reset_index()

df_days_repurchase_subsegment_2018["diff_order_purchase"] = df_days_repurchase_subsegment_2018["last_order_purchase"].values - df_days_repurchase_subsegment_2018["first_order_purchase"].values

df_days_repurchase_subsegment_2018["diff_order_purchase"] = df_days_repurchase_subsegment_2018["diff_order_purchase"].round(0)

In [13]:
configure_plotly_browser_state()
trace0 = go.Bar(
    x=df_days_repurchase_subsegment_2018["sub_segment"].values,
    y=df_days_repurchase_subsegment_2018["diff_order_purchase"].values,
    marker=dict(
        color=['rgba(36,123,160,1)', 
               'rgba(75,147,177,1)',
               'rgba(112,193,179,1)', 
               'rgba(138,204,192,1)',
               'rgba(243,255,189,1)',
               'rgba(247,255,213,1)',
               'rgba(255,22,84,1)']),
)

data = [trace0]

layout = go.Layout(
    title='Avg days between first and last purchase',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [14]:
period_2017, df_clustering_2017 = clustering_customers(df, "2017-12-31")

df_clustering_2017.head()

customer_unique_id  first_order_purchase  \
0  0000f46a3911fa3c0805444483337064                   296   
1  0000f6ccb0745a6a4b88665a16c9f078                    80   
2  0004aac84e0df4da2b147fca70cf8255                    47   
3  0005e1862207bf6ccc02e4228effd9a0                   302   
4  0006fdc98a402fceb4eb0ee528f6a8d4                   166   

   last_order_purchase  order_purchase_qty  payment_value_sum  \
0                  296                   1              86.22   
1                   80                   1              43.62   
2                   47                   1             196.89   
3                  302                   1             150.12   
4                  166                   1              29.00   

   payment_value_mean segment                sub_segment  new_customer  
0               86.22    cold     cold_low_payment_value             0  
1               43.62  active   active_low_payment_value             1  
2              196.89  active  active_high_payment_value             1  
3              150.12    cold    cold_high_payment_value             0  
4               29.00     hot      hot_low_payment_value             1